# Pruebas para verificar el cumplimiento del objetivo C

## Configuración Inicial y Autenticación

In [1]:
from utils import configure_headers


# configure the headers
configure_headers()

ENDPOINTS_TO_TEST = [
    # search
    "https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album",
    # get album
    "https://api.spotify.com/v1/albums/4aawyAB9vmqN3uQ7FjRGTy",
    # get artist
    "https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg",
    # get playlist
    "https://api.spotify.com/v1/playlists/3cEYpjA9oz9GiPac4AsH4n",
    # get track
    "https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl",
    # TODO Puedes añadir más endpoints aquí
]

✅ Token loaded.


# Objetivo C - Prueba de latencia con Red de 10 Mbps

Hacemos las pruebas base para cumplir con el objetivo C, usando la conexión de 10 Mbps. Vamos a probar todos los endpoints que declaramos que son los servicios a estudiar, y vamos a tomar todas las mediciones para luego poder responder a las métricas establecidas.

In [2]:
from utils import simulate_concurrent_users


simulate_concurrent_users(
    num_users=30,
    num_requests_per_user=30,
    endpoints_to_test=ENDPOINTS_TO_TEST
)


--- Simulation started with 30 concurrent users ---
Each user will do 30 requests.

✴️ Usuario 1 - Starting requests to https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album. - Test started at 1748096611.8808424
✴️ Usuario 2 - Starting requests to https://api.spotify.com/v1/albums/4aawyAB9vmqN3uQ7FjRGTy. - Test started at 1748096611.9046414
✴️ Usuario 3 - Starting requests to https://api.spotify.com/v1/artists/0TnOYISbd1XYRBk9myaseg. - Test started at 1748096611.930968
✴️ Usuario 4 - Starting requests to https://api.spotify.com/v1/playlists/3cEYpjA9oz9GiPac4AsH4n. - Test started at 1748096611.9851468
✴️ Usuario 5 - Starting requests to https://api.spotify.com/v1/tracks/11dFghVXANMlKmJXsNCbNl. - Test started at 1748096612.0156896
✴️ Usuario 6 - Starting requests to https://api.spotify.com/v1/search?q=remaster%2520track%3ADoxy%2520artist%3AMiles%2520Davis&type=album. - Test started at 1748096612.0786114
✴️ Usuario 7 - Starting requests to 

## Procesa los resultados

In [3]:
from utils import process_concurrent_results

# process the results
df_global_results, df_individual_results, df_by_endpoint = process_concurrent_results()


--- Resumen General de la Simulación Concurrente ---
Total de usuarios simulados: 30
Peticiones totales realizadas: 900
Peticiones exitosas totales: 177
Errores 4xx totales: 723
Errores 5xx totales: 0
Tasa de éxito global: 19.67%
Tasa de error 4xx global: 80.33%
Tasa de error 5xx global: 0.00%
Latencia promedio: 0.319


In [4]:
print('\nGlobal results\n')
df_global_results


Global results



,user_id,endpoint,total_requests,successful_requests,error_4xx_requests,error_5xx_requests,total_response_time,latencies,errors_occurred,test_duration
0,2,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,30,0,30,0,8.680045,"[0.2746157646179199, 0.2862715721130371, 0.413...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.704804
1,3,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,0,30,0,8.797691,"[0.2760763168334961, 0.2830390930175781, 0.413...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.817925
2,5,https://api.spotify.com/v1/tracks/11dFghVXANMl...,30,0,30,0,8.761015,"[0.264967679977417, 0.2885868549346924, 0.4450...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.784019
3,8,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,0,30,0,8.750155,"[0.294797420501709, 0.27727460861206055, 0.362...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.773198
4,10,https://api.spotify.com/v1/tracks/11dFghVXANMl...,30,0,30,0,8.683316,"[0.2877321243286133, 0.2740187644958496, 0.273...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.710526
5,7,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,30,0,30,0,8.925084,"[0.30202460289001465, 0.26990270614624023, 0.3...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.946982
6,12,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,30,0,30,0,8.754899,"[0.28330540657043457, 0.29024434089660645, 0.4...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.780637
7,13,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,0,30,0,8.762463,"[0.29169225692749023, 0.2918558120727539, 0.38...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.786189
8,17,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,30,0,30,0,8.876372,"[0.27548933029174805, 0.29442834854125977, 0.4...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.902056
9,18,https://api.spotify.com/v1/artists/0TnOYISbd1X...,30,0,30,0,8.894434,"[0.2635509967803955, 0.2672090530395508, 0.398...","[429, 429, 429, 429, 429, 429, 429, 429, 429, ...",38.921914


In [5]:
print('\nIndividual results\n')
df_individual_results


Individual results



,user_ids,endpoints,status_codes,latencies,times,error_intervals
0,2,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,429,0.274616,1.748097e+09,NaN
1,2,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,429,0.286272,1.748097e+09,1.289721
2,2,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,429,0.413513,1.748097e+09,1.414167
3,2,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,429,0.273674,1.748097e+09,1.273822
4,2,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,429,0.302046,1.748097e+09,1.302413
...,...,...,...,...,...,...
895,26,https://api.spotify.com/v1/search?q=remaster%2...,200,0.557652,1.748097e+09,NaN
896,26,https://api.spotify.com/v1/search?q=remaster%2...,200,0.557331,1.748097e+09,NaN
897,26,https://api.spotify.com/v1/search?q=remaster%2...,429,0.280727,1.748097e+09,5.914397
898,26,https://api.spotify.com/v1/search?q=remaster%2...,429,0.263750,1.748097e+09,1.264192


In [6]:
print('\nResults by endpoint\n')
df_by_endpoint


Results by endpoint



,endpoint,total_requests,successful_requests,error_4xx_requests,error_5xx_requests,avg_latency,num_users,success_rate,error_4xx_rate,error_5xx_rate
0,https://api.spotify.com/v1/albums/4aawyAB9vmqN...,180,0,180,0,0.293909,6,0.000000,100.000000,0.0
1,https://api.spotify.com/v1/artists/0TnOYISbd1X...,180,0,180,0,0.293442,6,0.000000,100.000000,0.0
2,https://api.spotify.com/v1/playlists/3cEYpjA9o...,180,91,89,0,0.335597,6,50.555556,49.444444,0.0
3,https://api.spotify.com/v1/search?q=remaster%2...,180,86,94,0,0.372474,6,47.777778,52.222222,0.0
4,https://api.spotify.com/v1/tracks/11dFghVXANMl...,180,0,180,0,0.301782,6,0.000000,100.000000,0.0


## Guardamos los resultados base

In [7]:
df_global_results.to_csv('./files/obj-C/global_results.csv', index=False)
df_individual_results.to_csv('./files/obj-C/individual_results.csv', index=False)
df_by_endpoint.to_csv('./files/obj-C/endpoints_results.csv', index=False)